In [1]:
def ner(f,f1,a1):
    from simpletransformers.ner import NERModel
    import pandas as pd
    import logging
    import re
    from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
    from transformers import pipeline
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    import openpyxl
    from openpyxl import load_workbook
    logging.basicConfig(level=logging.DEBUG)
    transformers_logger = logging.getLogger('transformers')
    transformers_logger.setLevel(logging.WARNING)

    
   
    file="/host/Sahithi/myenv/input/input/"+f1
    with open(file, "r") as cur:
        text=cur.read()
        #preprocessing of data
    exceptions = {
            r'\d+mg': r'\g<0> |\g<1>',
            r'\d+:\d+': r'\g<0>',
            r'[\,\-\(\)]': r' \g<0> ',
            r'(?<!\d)\d+\.\d+(?!\d)': r'\g<0>',
            r'\d+/\d+': r'\g<0>',
            r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}': r'\g<0>',
            r'[A-Za-z]\d': r'\g<0>',
            r'[^\.\s]\.[^\.\s]': r'\g<0>',
            r'\d+\.\d+':r'\g<0>',
        }
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    processed_tokens = []

    a = ["COVID-19", "4/5Unable", "workstation:<ID", "patientÂ\x92s"]
    sentences = [] # List to store processed sentences

    for token in tokens:
        if token in a:
            processed_tokens.append(token)
        elif token not in exceptions and (":" in token or "," in token or "/" in token or "." in token or "<" in token or ">" in token or "-" in token):
            if re.match(r'\d+/\d+', token) or re.match(r'\d+:\d+', token):
                processed_tokens.append(token)
            elif re.match(r'(?<!\d)\d+\.\d+(?!\d)', token):
                processed_tokens.append(token)
            else:
                split_tokens = re.split(r'(:|,|/|\.|<|>|-)', token)
                processed_tokens.extend([t for t in split_tokens if t])
        elif token in exceptions:
            processed_tokens.append(exceptions[token])
        else:
            processed_tokens.append(token)

    # Iterate over sentences and append processed tokens
    current_sentence =""
    sentences=""
    for token in processed_tokens:
        current_sentence=current_sentence+" "+token
   
        if token.endswith('.') or token.endswith('!') or token.endswith('?'):
            sentences=sentences+" "+current_sentence
            current_sentence=""

    # Append the remaining tokens as the last sentence
    if current_sentence:
        sentences=sentences+current_sentence
    c=[]
    sentences1=sentences.replace('REMITTING MULTIPLE SCLEROSIS','remitting multiple sclerosis')
    
        #splitting the total sentences with maximum length as 350(it can be userdefined )
    def process_input(s, max_length):
        if len(s) <= max_length:
            c.append(s)
        else:
            while len(s) > 0:
                chunk = s[:max_length] 
                c.append(chunk)
                s = s[max_length:]


    max_length = 350
    process_input(sentences1, max_length)



    # Annotated file for each document
    a2="/host/Sahithi/myenv/output_entity/"+a1
    df = pd.read_csv(a2, delimiter='\n', header=None, quoting=3)
    annotated_text = df[0].tolist()
    a = []
    for i in annotated_text:
        b = i.split(' ')
        a.append(b)



    tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")
    model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    x=[]
    p=[]
    for i in c: #c is the chunks of the data with max limit 350
        p.extend(pipe(i))
    #predicted ones
    
#     for i in x:
#         p.extend(i)  #combining predictions dictioneries of the chunks into a list
    predicted_tokens = []
    for item in p:
        for word in item['word'].split():
            predicted_tokens.append([word,item['entity_group']])

    annotated_tokens = [[item[0],item[3]] for item in a] #making list of annotated_tokens and labels
    annotated_num=0
    for i in annotated_tokens:
        if(i[1]!='O'):
            annotated_num+=1 #counting the number of annotated tokens of the corresponding conll files


    tp = []
    tn = []
    fp = []
    fn = []

    for item in predicted_tokens:
        count = 0
        for items in annotated_tokens:
            if item[0] == items[0]  or items[0] in item[0].upper(): 
                count = 1
                if (item[1] == '0') & (items[1]=='O'):
                    tn.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == 'DISEASE') & (items[1]!='O'):
                    tp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == 'DISEASE') & (items[1]=='O'):
                    fp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == '0') & (items[1]!='O'):
                    fn.append([item,items])
                    annotated_tokens.remove(items)
                    break

        if count == 0:
            if item[1] == 'DISEASE':
                fp.append([[item,[item[0],'O']]])
            elif item[1] == '0':
                tn.append([[item,[item[0],'O']]])
    print(len(tp),len(fp),len(tn),len(fn))
    
    inscope=0
    flag=False
    anno=0
    if(len(tp)==0 or len(fp)==0 or len(tn)==0 ):
        print("somevalue is zer0")
          
    else:
         
        flag=True

        total=[]
        [total.extend(l) for l in (tp,tn,fp,fn)]

        token=[]
        predict=[]
        annot=[]
        for i in total:
            if(len(i)==2):
                token.append(i[0][0])
                predict.append(i[0][1])
                annot.append(i[1][1])
            elif(len(i)==1):
                for j in i:
                    token.append(j[0][0])
                    predict.append(j[0][1])
                    annot.append(j[1][1])

        output=pd.DataFrame({
                        'token':token,
                        'predictions':predict,
                        'annotated':annot,
                    })
        
        
        for i in total:   #finding the number of inscope tokens 
            if(len(i)==2):
                if(i[0][1]!='0' and i[1][1]!='O'):
                    inscope=inscope+1
                    print(i)
            elif len(i)==1:
                for j in i:
                    if(j[0][1]!='0' and j[1][1]!='O'):
                        inscope=inscope+1
                        print(j)
       
                        
        print("inscope:",inscope)
        
            
        
        
    return len(token),annotated_num,inscope,flag,len(tp),len(tn),len(fp),len(fn)

    

import os
files = os.listdir('/host/Sahithi/myenv/input/input/')
an_files=os.listdir('/host/Sahithi/myenv/output_entity/')
tokens=0
annotated_tokens=0
num_file=0
inscope_tokens=0
tp=0
tn=0
fp=0
fn=0
for an_file in an_files:
    f=os.path.splitext(os.path.basename(an_file))[0]
    for file in files:
        f1=os.path.splitext(os.path.basename(file))[0]
        if f1 in f:
            print(f1,f)
            print(file)
            t,an,ins,flag,t1,t2,t3,t4=ner(f1,file,an_file)
#             if(t1!=0 and t2!=0 and t3!=0 and t4!=0):
            if flag==True:
                tokens=tokens+t
                annotated_tokens=annotated_tokens+an
                tp=tp+t1
                tn=tn+t2
                fp=fp+t3
                fn=fn+t4
                inscope_tokens+=ins
                num_file+=1
                
                
            
            
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)  



batch_04_00011 annotator2_batch_04_00011_correct
batch_04_00011.txt


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


0 35 429 1
somevalue is zer0
batch_05_00176 annotator2_batch_05_00176_correct
batch_05_00176.txt


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


4 15 541 5
[['Multiple', 'DISEASE'], ['Multiple', 'I-Disease_Subtype']]
[['Sclerosis', 'DISEASE'], ['Sclerosis', 'I-Disease_Subtype']]
[['MS', 'DISEASE'], ['MS', 'B-Disease_Subtype']]
[['MS', 'DISEASE'], ['MS', 'I-Disease_Subtype']]
inscope: 4
batch_05_00177 annotator2_batch_05_00177_correct
batch_05_00177.txt


...

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


3 95 1418 5
[['MS', 'DISEASE'], ['MS', 'B-RRMS']]
[['multiple', 'DISEASE'], ['multiple', 'I-RRMS']]
[['sclerosis', 'DISEASE'], ['sclerosis', 'I-RRMS']]
inscope: 3
recent_00050 annotator2_recent_00050_correct
recent_00050.txt


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /alvaroalon2/biobert_diseases_ner/resolve/main/config.json HTTP/1.1" 200 0


6 57 1930 7
[['multiple', 'DISEASE'], ['multiple', 'B-RRMS']]
[['sclerosis', 'DISEASE'], ['sclerosis', 'I-RRMS']]
[['##RMS', 'DISEASE'], ['MS', 'B-RRMS']]
[['Multiple', 'DISEASE'], ['Multiple', 'B-RRMS']]
[['Sclerosis', 'DISEASE'], ['Sclerosis', 'I-RRMS']]
[['MS', 'DISEASE'], ['MS', 'B-RRMS']]
inscope: 6
.ipynb_checkpoints .ipynb_checkpoints
.ipynb_checkpoints


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:249                                                                                  │
│ in ner:18                                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IsADirectoryError: [Errno 21] Is a directory: '/host/Sahithi/myenv/input/input/.ipynb_checkpoints'

In [3]:
print(num_file,inscope_tokens,annotated_tokens)

405 1839 5201


In [5]:
print("number of  tokens:",tokens  )
print("number of annotated tokens:",annotated_tokens)
print("inscope _tokens:",inscope_tokens)
print("true positives:",tp)
print("true negatives:",tn)
print("false positives:",fp)
print("false negatives:",fn)


precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)  

print("precision:",precision*100)
print("recall:",recall*100)
print("f1 score:",f1*100)

number of  tokens: 358006
number of annotated tokens: 5201
inscope _tokens: 1839
true positives: 1839
true negatives: 334370
false positives: 19041
false negatives: 2756
precision: 8.807471264367816
recall: 40.02176278563656
f1 score: 14.437684003925416
